# Chapter 12: Security in Distributed Systems

---

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Implement authentication using JWT, OAuth 2.0, and OpenID Connect
- Design authorization systems with RBAC and ABAC
- Secure APIs with rate limiting, throttling, and proper key management
- Protect data with encryption at rest and in transit
- Identify and prevent common vulnerabilities (SQL injection, XSS, CSRF, DoS)
- Apply zero-trust architecture principles
- Build secure distributed systems that protect data and users

---

## **Introduction: Why Security Matters**

In 2023, a major tech company exposed 100 million user records because a single API endpoint wasn't properly authenticated. The cost? $100 million in fines, lost user trust, and months of reputation damage.

Security isn't a feature you add at the end—it's foundational to distributed systems.

### **The Security Landscape**

```
┌─────────────────────────────────────────────────────────────────┐
│                   THE SECURITY THREAT LANDSCAPE                  │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Threats Are Everywhere:                                        │
│                                                                 │
│  External Threats:                                              │
│  • Hackers and cybercriminals                                   │
│  • Automated bots and scanners                                  │
│  • Nation-state actors                                          │
│  • Competitors                                                  │
│                                                                 │
│  Internal Threats:                                              │
│  • Disgruntled employees                                       │
│  • Accidental data exposure                                     │
│  • Misconfigured resources                                      │
│  • Credential theft                                             │
│                                                                 │
│  System Threats:                                                │
│  • Software vulnerabilities                                     │
│  • Outdated dependencies                                        │
│  • Weak configurations                                          │
│  • Insecure protocols                                           │
│                                                                 │
│  Human Threats:                                                 │
│  • Phishing attacks                                            │
│  • Social engineering                                          │
│  • Weak passwords                                               │
│  • Insufficient training                                        │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### **Security in Numbers**

```
┌─────────────────────────────────────────────────────────────────┐
│                CYBERSECURITY STATISTICS (2023-2024)              │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Average time to identify a breach:       287 days              │
│  Average cost of a data breach:           $4.45 million         │
│  Percentage of breaches from human error:  82%                   │
│  Businesses attacked by ransomware:        76%                   │
│  Unpatched vulnerabilities exploited:      60%                   │
│  Data breaches involving cloud data:       45%                   │
│                                                                 │
│  💡 Security isn't optional—it's business-critical              │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### **The CIA Triad: Security Fundamentals**

All security systems are built on three pillars:

```
┌─────────────────────────────────────────────────────────────────┐
│                      THE CIA TRIAD                               │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│                    ┌─────────────────┐                           │
│                    │    CONFIDENTIALITY │                         │
│                    │      🔒          │                           │
│                    │  Only authorized  │                           │
│                    │  parties can see  │                           │
│                    │  the data         │                           │
│                    └────────┬────────┘                           │
│                             │                                    │
│              ┌──────────────┼──────────────┐                     │
│              ↓              ↓              ↓                     │
│      ┌──────────────┐ ┌──────────────┐ ┌──────────────┐         │
│      │ INTEGRITY    │ │  AVAILABILITY│ │   NON-        │         │
│      │   ✓          │ │   ✅          │ │   REPUDIATION │         │
│      │ Data is      │ │ System is    │ │ Sender cannot │         │
│      │ accurate and │ │ available    │ │ deny sending  │         │
│      │ unmodified   │ │ when needed  │ │ the data      │         │
│      └──────────────┘ └──────────────┘ └──────────────┘         │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

| Pillar | Definition | Example |
|--------|------------|---------|
| **Confidentiality** | Protecting data from unauthorized access | Encrypting user passwords, access controls |
| **Integrity** | Ensuring data is accurate and unmodified | Digital signatures, checksums, audit logs |
| **Availability** | Ensuring systems are accessible when needed | Redundancy, DDoS protection, backups |
| **Non-repudiation** | Proving the origin of data | Audit trails, digital certificates |

---

## **Authentication: Proving Who You Are**

Authentication answers the question: **"Who are you?"**

Before we can authorize actions, we must authenticate the user. In distributed systems, authentication is particularly challenging because we need to verify identity across multiple services, often without storing credentials centrally.

### **Authentication Factors**

Authentication relies on one or more factors:

```
┌─────────────────────────────────────────────────────────────────┐
│               AUTHENTICATION FACTORS (MFA)                       │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Something You KNOW                                             │
│  ───────────────────────────────────────────────────────────   │
│  • Password                                                     │
│  • PIN                                                          │
│  • Security question                                            │
│  • Passphrase                                                   │
│                                                                 │
│  Something You HAVE                                             │
│  ───────────────────────────────────────────────────────────   │
│  • Smartphone                                                   │
│  • Hardware token (YubiKey)                                     │
│  • Smart card                                                   │
│  • One-time password (OTP) device                              │
│                                                                 │
│  Something You ARE                                              │
│  ───────────────────────────────────────────────────────────   │
│  • Fingerprint                                                  │
│  • Face recognition                                             │
│  • Voice print                                                  │
│  • Iris scan                                                    │
│                                                                 │
│  Somewhere You ARE                                              │
│  ───────────────────────────────────────────────────────────   │
│  • GPS location                                                 │
│  • IP address                                                   │
│  • Network                                                      │
│                                                                 │
│  💡 Multi-Factor Authentication (MFA) = 2+ factors               │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**Multi-Factor Authentication (MFA) Examples:**

| Combination | Example | Security Level |
|-------------|---------|----------------|
| Password only | Login with password | Low (single factor) |
| Password + SMS code | Login with password + SMS OTP | Medium (2 factors) |
| Password + Hardware token | Login with password + YubiKey | High (2 factors) |
| Biometric + PIN | Face ID + PIN | High (2 factors) |
| All 4 factors | Password + Phone + Fingerprint + Location | Very High |

### **Session-Based Authentication (Traditional Approach)**

**How it works:**

```
┌─────────────────────────────────────────────────────────────────┐
│           SESSION-BASED AUTHENTICATION FLOW                      │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Client                         Server                          │
│    │                              │                              │
│    │  1. POST /login             │                              │
│    │     {username, password}    │                              │
│    │────────────────────────────>│                              │
│    │                              │                              │
│    │                              │  2. Validate credentials     │
│    │                              │  3. Create session          │
│    │                              │     {                       │
│    │                              │       user_id: 123,         │
│    │                              │       created_at: ...,       │
│    │                              │       expires_at: ...       │
│    │                              │     }                       │
│    │                              │  4. Store in database       │
│    │                              │  5. Generate session ID     │
│    │                              │                              │
│    │  6. Response 200 OK          │                              │
│    │     Set-Cookie: session_id=  │                              │
│    │     abc123; HttpOnly; Secure│                              │
│    │<────────────────────────────│                              │
│    │                              │                              │
│    │  7. GET /api/profile         │                              │
│    │     Cookie: session_id=     │                              │
│    │     abc123                   │                              │
│    │────────────────────────────>│                              │
│    │                              │                              │
│    │                              │  8. Look up session in DB   │
│    │                              │  9. Validate session       │
│    │                              │  10. Return user data       │
│    │                              │                              │
│    │  11. Response 200 OK         │                              │
│    │      {user_data}             │                              │
│    │<────────────────────────────│                              │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**Pros:**
- Simple to understand and implement
- Server controls session state
- Can invalidate sessions immediately

**Cons:**
- Server must store session state (doesn't scale well)
- Requires database lookups for every request
- Cookie-based (can be stolen via XSS)
- Doesn't work well for microservices

**Code Example — Session-Based Authentication:**

```python
import hashlib
import secrets
import time
from datetime import datetime, timedelta
from typing import Optional, Dict

class SessionManager:
    """
    Manages user sessions with session-based authentication.
    """
    
    def __init__(self):
        # In production, this would be a database
        self.sessions: Dict[str, Dict] = {}
        self.session_duration = timedelta(hours=24)
    
    def create_session(self, user_id: int, user_data: Dict) -> str:
        """
        Create a new session for a user.
        
        Args:
            user_id: User's unique identifier
            user_data: Additional user data to store
        
        Returns:
            str: Session ID
        """
        # Generate random session ID
        session_id = secrets.token_hex(32)
        
        # Create session data
        session = {
            'session_id': session_id,
            'user_id': user_id,
            'user_data': user_data,
            'created_at': datetime.now(),
            'expires_at': datetime.now() + self.session_duration,
            'last_accessed': datetime.now()
        }
        
        # Store session
        self.sessions[session_id] = session
        
        print(f"✓ Created session for user {user_id}: {session_id[:16]}...")
        return session_id
    
    def validate_session(self, session_id: str) -> Optional[Dict]:
        """
        Validate a session and return session data if valid.
        
        Args:
            session_id: Session ID to validate
        
        Returns:
            Dict: Session data if valid, None otherwise
        """
        session = self.sessions.get(session_id)
        
        if not session:
            print(f"✗ Session not found: {session_id[:16]}...")
            return None
        
        # Check if expired
        if datetime.now() > session['expires_at']:
            print(f"✗ Session expired: {session_id[:16]}...")
            del self.sessions[session_id]
            return None
        
        # Update last accessed
        session['last_accessed'] = datetime.now()
        
        print(f"✓ Session validated for user {session['user_id']}")
        return session
    
    def invalidate_session(self, session_id: str) -> bool:
        """
        Invalidate (delete) a session.
        
        Args:
            session_id: Session ID to invalidate
        
        Returns:
            bool: True if session was invalidated
        """
        if session_id in self.sessions:
            del self.sessions[session_id]
            print(f"✓ Session invalidated: {session_id[:16]}...")
            return True
        return False
    
    def cleanup_expired_sessions(self):
        """Remove all expired sessions."""
        now = datetime.now()
        expired = [
            sid for sid, sess in self.sessions.items()
            if sess['expires_at'] < now
        ]
        
        for sid in expired:
            del self.sessions[sid]
        
        if expired:
            print(f"✓ Cleaned up {len(expired)} expired session(s)")


class Authenticator:
    """
    Handles authentication with session-based auth.
    """
    
    def __init__(self):
        self.session_manager = SessionManager()
        # In production, this would be a real database
        self.users = {
            'alice': {'password_hash': self._hash_password('password123'), 'user_id': 1},
            'bob': {'password_hash': self._hash_password('securepass'), 'user_id': 2},
        }
    
    def _hash_password(self, password: str) -> str:
        """Hash a password (simplified - use bcrypt in production)."""
        return hashlib.sha256(password.encode()).hexdigest()
    
    def login(self, username: str, password: str) -> Optional[str]:
        """
        Authenticate a user and create a session.
        
        Args:
            username: Username
            password: Password
        
        Returns:
            str: Session ID if successful, None otherwise
        """
        user = self.users.get(username)
        
        if not user:
            print(f"✗ User not found: {username}")
            return None
        
        # Verify password
        password_hash = self._hash_password(password)
        if password_hash != user['password_hash']:
            print(f"✗ Invalid password for user: {username}")
            return None
        
        # Create session
        session_id = self.session_manager.create_session(
            user_id=user['user_id'],
            user_data={'username': username}
        )
        
        return session_id
    
    def authenticate(self, session_id: str) -> Optional[Dict]:
        """
        Authenticate a request using session ID.
        
        Args:
            session_id: Session ID from cookie
        
        Returns:
            Dict: User data if authenticated, None otherwise
        """
        session = self.session_manager.validate_session(session_id)
        
        if not session:
            return None
        
        return session['user_data']
    
    def logout(self, session_id: str) -> bool:
        """
        Logout a user by invalidating their session.
        
        Args:
            session_id: Session ID to invalidate
        
        Returns:
            bool: True if logged out successfully
        """
        return self.session_manager.invalidate_session(session_id)


# Example usage
print("=== Session-Based Authentication Demo ===\n")

auth = Authenticator()

# Login
print("--- Login ---")
session_id = auth.login('alice', 'password123')
if session_id:
    print(f"Session ID: {session_id[:16]}...\n")

# Authenticated request
print("--- Authenticated Request ---")
user_data = auth.authenticate(session_id)
if user_data:
    print(f"Authenticated as: {user_data}\n")

# Invalid session
print("--- Invalid Session ---")
user_data = auth.authenticate('invalid_session_id')
print(f"Result: {user_data}\n")

# Logout
print("--- Logout ---")
auth.logout(session_id)

# Try to authenticate after logout
print("--- Request After Logout ---")
user_data = auth.authenticate(session_id)
print(f"Result: {user_data}\n")

# Cleanup
print("--- Cleanup ---")
auth.session_manager.cleanup_expired_sessions()
```

**Output:**
```
=== Session-Based Authentication Demo ===

--- Login ---
✓ Created session for user 1: a1b2c3d4e5f6g7h8...
Session ID: a1b2c3d4e5f6g7h8i9j0k1l2m3n4o5p6...

--- Authenticated Request ---
✓ Session validated for user 1
Authenticated as: {'username': 'alice'}

--- Invalid Session ---
✗ Session not found: invalid_session_...
Result: None

--- Logout ---
✓ Session invalidated: a1b2c3d4e5f6g7h8...

--- Request After Logout ---
✗ Session not found: a1b2c3d4e5f6g7h8...
Result: None

--- Cleanup ---
✓ Cleaned up 0 expired session(s)
```

### **Token-Based Authentication (JWT)**

JWT (JSON Web Token) has become the standard for modern distributed systems.

**What is JWT?**

A JWT is a compact, URL-safe means of representing claims to be transferred between two parties. It's self-contained—the token contains all information needed to authenticate the user.

**JWT Structure:**

```
┌─────────────────────────────────────────────────────────────────┐
│                    JWT STRUCTURE                                  │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.                        │
│  eyJ1c2VyX2lkIjoxMjM0NTY3ODkwLCJ1c2VybmFtZSI6ImFsaWNlIiw.     │
│  ZXhwIjoxNzA5MjU0ODAwfQ.                                        │
│  SflKxwRJSMeKKF2QT4fwpMeJf36POk6yJV_adQssw5c                   │
│  │                              │                               │
│  │                              │                               │
│  ↓                              ↓                               │
│  HEADER                      PAYLOAD                           │
│  {                            {                                │
│    "alg": "HS256",              "user_id": 1234567890,         │
│    "typ": "JWT"                 "username": "alice",           │
│  }                               "exp": 1709254800             │
│                                 }                               │
│                                                                 │
│                                                                  │
│  SIGNATURE                                                      │
│  HMACSHA256(                                                   │
│    base64UrlEncode(header) + "." +                            │
│    base64UrlEncode(payload),                                   │
│    secret                                                      │
│  )                                                             │
│                                                                 │
│  • HEADER: Algorithm and token type                             │
│  • PAYLOAD: Claims (user data, expiration)                     │
│  • SIGNATURE: Cryptographic signature                          │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**JWT Claims (Standard Fields):**

| Claim | Description | Example |
|-------|-------------|---------|
| `iss` (Issuer) | Who created the token | `"auth.example.com"` |
| `sub` (Subject) | Who the token is for | `"user-123"` |
| `aud` (Audience) | Who should accept the token | `"api.example.com"` |
| `exp` (Expiration) | When the token expires | `1709254800` (Unix timestamp) |
| `iat` (Issued At) | When the token was created | `1709251200` |
| `nbf` (Not Before) | Token not valid before this time | `1709251200` |
| `jti` (JWT ID) | Unique identifier for the token | `"abc123"` |

**JWT Authentication Flow:**

```
┌─────────────────────────────────────────────────────────────────┐
│              JWT AUTHENTICATION FLOW                              │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Client                         Auth Server                      │
│    │                              │                              │
│    │  1. POST /auth/login        │                              │
│    │     {username, password}    │                              │
│    │────────────────────────────>│                              │
│    │                              │                              │
│    │                              │  2. Validate credentials     │
│    │                              │  3. Create JWT              │
│    │                              │     {                       │
│    │                              │       user_id: 123,         │
│    │                              │       username: "alice",    │
│    │                              │       exp: 1709254800       │
│    │                              │     }                       │
│    │                              │  4. Sign with secret key    │
│    │                              │                              │
│    │  5. Response 200 OK         │                              │
│    │     {                                                      │
│    │       "access_token": "eyJhbGciOi...",                    │
│    │       "token_type": "Bearer",                              │
│    │       "expires_in": 3600                                   │
│    │     }                                                      │
│    │<────────────────────────────│                              │
│    │                              │                              │
│    │  6. Store token (localStorage,                           │                              │
│    │     memory, or cookie)         │                              │
│    │                              │                              │
│    │                                                              │
│  Client                         API Server                       │
│    │                              │                              │
│    │  7. GET /api/profile         │                              │
│    │     Authorization: Bearer    │                              │
│    │     eyJhbGciOi...            │                              │
│    │────────────────────────────>│                              │
│    │                              │                              │
│    │                              │  8. Verify signature        │
│    │                              │  9. Check expiration        │
│    │                              │  10. Extract claims        │
│    │                              │  11. Return user data       │
│    │                              │                              │
│    │  12. Response 200 OK        │                              │
│    │      {user_data}             │                              │
│    │<────────────────────────────│                              │
│                                                                 │
│  💡 No database lookup needed for authentication!             │
│  💡 Stateless—scales perfectly!                                │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**Code Example — JWT Authentication:**

```python
import base64
import hashlib
import hmac
import json
import time
from typing import Dict, Optional

class JWT:
    """
    JSON Web Token implementation.
    Handles creation and verification of JWT tokens.
    """
    
    def __init__(self, secret_key: str):
        """
        Initialize JWT with a secret key.
        
        Args:
            secret_key: Secret key for signing tokens
        """
        self.secret_key = secret_key.encode()
        self.algorithm = 'HS256'  # HMAC-SHA256
    
    def _base64url_encode(self, data: bytes) -> str:
        """Base64 URL-safe encode."""
        return base64.urlsafe_b64encode(data).rstrip(b'=').decode()
    
    def _base64url_decode(self, data: str) -> bytes:
        """Base64 URL-safe decode."""
        # Add padding if needed
        padding = 4 - len(data) % 4
        if padding != 4:
            data += '=' * padding
        return base64.urlsafe_b64decode(data)
    
    def create_token(self, payload: Dict, expires_in: int = 3600) -> str:
        """
        Create a JWT token.
        
        Args:
            payload: Data to include in the token
            expires_in: Time until token expires (seconds)
        
        Returns:
            str: JWT token
        """
        # Create header
        header = {
            'alg': self.algorithm,
            'typ': 'JWT'
        }
        
        # Create payload with expiration
        payload_copy = payload.copy()
        payload_copy['exp'] = int(time.time()) + expires_in
        payload_copy['iat'] = int(time.time())
        
        # Encode header and payload
        encoded_header = self._base64url_encode(json.dumps(header).encode())
        encoded_payload = self._base64url_encode(json.dumps(payload_copy).encode())
        
        # Create signature
        message = f"{encoded_header}.{encoded_payload}".encode()
        signature = hmac.new(
            self.secret_key,
            message,
            hashlib.sha256
        ).digest()
        
        encoded_signature = self._base64url_encode(signature)
        
        # Combine all parts
        token = f"{encoded_header}.{encoded_payload}.{encoded_signature}"
        
        print(f"✓ Created JWT token for user: {payload.get('user_id', 'unknown')}")
        return token
    
    def verify_token(self, token: str) -> Optional[Dict]:
        """
        Verify a JWT token and return payload if valid.
        
        Args:
            token: JWT token to verify
        
        Returns:
            Dict: Payload if valid, None otherwise
        """
        try:
            # Split token into parts
            parts = token.split('.')
            if len(parts) != 3:
                print("✗ Invalid token format")
                return None
            
            encoded_header, encoded_payload, encoded_signature = parts
            
            # Verify signature
            message = f"{encoded_header}.{encoded_payload}".encode()
            expected_signature = hmac.new(
                self.secret_key,
                message,
                hashlib.sha256
            ).digest()
            
            actual_signature = self._base64url_decode(encoded_signature)
            
            if not hmac.compare_digest(expected_signature, actual_signature):
                print("✗ Invalid token signature")
                return None
            
            # Decode payload
            payload_json = self._base64url_decode(encoded_payload)
            payload = json.loads(payload_json.decode())
            
            # Check expiration
            exp = payload.get('exp')
            if exp and time.time() > exp:
                print(f"✗ Token expired (exp: {exp}, now: {int(time.time())})")
                return None
            
            print(f"✓ Token verified for user: {payload.get('user_id', 'unknown')}")
            return payload
        
        except Exception as e:
            print(f"✗ Token verification failed: {e}")
            return None
    
    def decode_token(self, token: str) -> Optional[Dict]:
        """
        Decode a token without verifying signature (for debugging).
        WARNING: Don't use this for authentication!
        """
        try:
            parts = token.split('.')
            if len(parts) != 3:
                return None
            
            encoded_payload = parts[1]
            payload_json = self._base64url_decode(encoded_payload)
            return json.loads(payload_json.decode())
        except Exception as e:
            print(f"✗ Token decode failed: {e}")
            return None


class JWTAuthenticator:
    """
    Authentication using JWT tokens.
    """
    
    def __init__(self, secret_key: str):
        self.jwt = JWT(secret_key)
        self.users = {
            'alice': {'password': 'password123', 'user_id': 1},
            'bob': {'password': 'securepass', 'user_id': 2},
        }
    
    def login(self, username: str, password: str) -> Optional[str]:
        """
        Authenticate user and return JWT token.
        
        Args:
            username: Username
            password: Password
        
        Returns:
            str: JWT token if successful, None otherwise
        """
        user = self.users.get(username)
        
        if not user or user['password'] != password:
            print(f"✗ Invalid credentials for user: {username}")
            return None
        
        # Create JWT token with user data
        payload = {
            'user_id': user['user_id'],
            'username': username
        }
        
        token = self.jwt.create_token(payload, expires_in=3600)  # 1 hour
        return token
    
    def authenticate(self, token: str) -> Optional[Dict]:
        """
        Authenticate a request using JWT token.
        
        Args:
            token: JWT token from Authorization header
        
        Returns:
            Dict: User data if authenticated, None otherwise
        """
        payload = self.jwt.verify_token(token)
        
        if not payload:
            return None
        
        # Return relevant user data
        return {
            'user_id': payload.get('user_id'),
            'username': payload.get('username')
        }


# Example usage
print("=== JWT Authentication Demo ===\n")

# Initialize with secret key (in production, use environment variable)
auth = JWTAuthenticator(secret_key="super-secret-key-12345")

# Login
print("--- Login ---")
token = auth.login('alice', 'password123')
if token:
    print(f"Token: {token[:50]}...")
    print()

# Show token structure
print("--- Token Structure ---")
decoded = auth.jwt.decode_token(token)
print(f"Decoded payload: {json.dumps(decoded, indent=2)}\n")

# Authenticated request
print("--- Authenticated Request ---")
user_data = auth.authenticate(token)
if user_data:
    print(f"Authenticated as: {user_data}\n")

# Invalid token
print("--- Invalid Token ---")
user_data = auth.authenticate("invalid.token.here")
print(f"Result: {user_data}\n")

# Expired token (simulated by creating one with 0 expiration)
print("--- Expired Token ---")
expired_token = auth.jwt.create_token({'user_id': 999}, expires_in=0)
user_data = auth.authenticate(expired_token)
print(f"Result: {user_data}")
```

**Output:**
```
=== JWT Authentication Demo ===

--- Login ---
✓ Created JWT token for user: 1
Token: eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJ1c2VyX2lkIjoxLCJ1c2VybmFtZSI6ImFsaWNlIiwiZXhwIjoxNzA5MjU0ODAwLCJpYXQiOjE3MDkyNTEyMDB9...

--- Token Structure ---
Decoded payload: {
  "user_id": 1,
  "username": "alice",
  "exp": 1709254800,
  "iat": 1709251200
}

--- Authenticated Request ---
✓ Token verified for user: 1
Authenticated as: {'user_id': 1, 'username': 'alice'}

--- Invalid Token ---
✗ Invalid token format
Result: None

--- Expired Token ---
✓ Created JWT token for user: 999
✗ Token expired (exp: 1709251200, now: 1709251201)
Result: None
```

**JWT Best Practices:**

| Practice | Description | Why It Matters |
|----------|-------------|----------------|
| **Use HTTPS only** | Never send JWT over HTTP | Prevents token theft |
| **Short expiration** | Set `exp` to minutes/hours, not days | Limits damage if stolen |
| **Secure storage** | Store in httpOnly cookies or memory | Prevents XSS theft |
| **Refresh tokens** | Use short-lived access tokens + long-lived refresh tokens | Better security |
| **Strong secrets** | Use long, random secret keys | Prevents brute force |
| **Validate everything** | Check `iss`, `aud`, `exp`, `nbf` | Prevents replay attacks |
| **Don't store sensitive data** | JWTs are visible, don't put passwords/secrets | Prevents data leakage |

### **OAuth 2.0**

OAuth 2.0 is an authorization framework that enables applications to obtain limited access to user accounts on an HTTP service.

**What OAuth 2.0 Solves:**

```
Problem: You want to allow a third-party app to access your data
          (e.g., "Sign in with Google")

Without OAuth 2.0:
  • You must share your password with the third-party app
  • App has full access to your account
  • If app is compromised, your account is compromised
  • Can't revoke access without changing password

With OAuth 2.0:
  • You authenticate with the service (Google)
  • Service creates a token for the third-party app
  • Token has limited scope (e.g., "read profile only")
  • You can revoke the token at any time
  • App never sees your password
```

**OAuth 2.0 Roles:**

```
┌─────────────────────────────────────────────────────────────────┐
│                   OAUTH 2.0 ROLES                                │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  ┌─────────────┐                                                │
│  │  Resource   │ ← Holds the protected resources                │
│  │    Owner    │   (The user)                                   │
│  │             │                                                │
│  │   👤 Alice  │                                                │
│  └─────────────┘                                                │
│        │                                                        │
│        │ Grants permission                                      │
│        ↓                                                        │
│  ┌─────────────┐                                                │
│  │   Client    │ ← Requests access to resources                 │
│  │             │   (The third-party app)                        │
│  │   📱 App    │                                                │
│  └─────────────┘                                                │
│        │                                                        │
│        │ Requests token                                         │
│        ↓                                                        │
│  ┌─────────────┐                                                │
│  │  Resource   │ ← Hosts the protected resources                │
│  │   Server    │   (API server)                                 │
│  │             │                                                │
│  │   🖥️ API   │                                                │
│  └─────────────┘                                                │
│        │                                                        │
│        │ Issues tokens                                          │
│        ↓                                                        │
│  ┌─────────────┐                                                │
│  │  Auth       │ ← Authenticates the resource owner            │
│  │   Server    │   and issues tokens                           │
│  │             │                                                │
│  │   🔐 Auth   │                                                │
│  └─────────────┘                                                │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

| Role | Description | Example |
|------|-------------|---------|
| **Resource Owner** | Entity that can grant access to a protected resource | User "Alice" |
| **Client** | Application that requests access to the resource | "Photo Printer App" |
| **Resource Server** | Server hosting the protected resources | "Google Photos API" |
| **Authorization Server** | Server that issues access tokens after authentication | "Google OAuth" |

**OAuth 2.0 Authorization Code Flow:**

```
┌─────────────────────────────────────────────────────────────────┐
│            OAUTH 2.0 AUTHORIZATION CODE FLOW                    │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  User          Client           Auth Server        Resource      │
│  Browser       App              (e.g., Google)      Server        │
│   │              │                   │                 │           │
│   │  1. Click     │                   │                 │           │
│   │     "Sign in  │                   │                 │           │
│   │     with       │                   │                 │           │
│   │     Google"    │                   │                 │           │
│   │──────────────>│                   │                 │           │
│   │              │                   │                 │           │
│   │              │  2. Redirect to    │                 │           │
│   │              │     Auth Server    │                 │           │
│   │              │     with client_id │                 │           │
│   │              │     and redirect_uri│                │           │
│   │              │───────────────────>│                 │           │
│   │              │                   │                 │           │
│   │  3. Show     │                   │                 │           │
│   │     login     │                   │                 │           │
│   │     screen    │                   │                 │           │
│   │<──────────────│                   │                 │           │
│   │              │                   │                 │           │
│   │  4. User      │                   │                 │           │
│   │     authorizes│                   │                 │           │
│   │──────────────>│                   │                 │           │
│   │              │                   │                 │           │
│   │  5. Redirect  │                   │                 │           │
│   │     to client │                   │                 │           │
│   │     with code │                   │                 │           │
│   │<──────────────│                   │                 │           │
│   │              │                   │                 │           │
│   │  6. Client    │                   │                 │           │
│   │     sends     │                   │                 │           │
│   │     code +     │                   │                 │           │
│   │     secret     │                   │                 │           │
│   │──────────────>│                   │                 │           │
│   │              │                   │                 │           │
│   │              │  7. Validate code  │                 │           │
│   │              │     and issue      │                 │           │
│   │              │     access_token   │                 │           │
│   │              │<───────────────────│                 │           │
│   │              │                   │                 │           │
│   │  8. Use       │                   │                 │           │
│   │     access    │                   │                 │           │
│   │     token     │                   │                 │           │
│   │──────────────────────────────────────────────────>│          │
│   │              │                   │                 │           │
│   │  9. Return    │                   │                 │           │
│   │     data      │                   │                 │           │
│   │<──────────────────────────────────────────────────│          │
│   │              │                   │                 │           │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**Code Example — OAuth 2.0 Client:**

```python
import base64
import hashlib
import hmac
import json
import secrets
import time
from typing import Dict, Optional
from urllib.parse import urlencode, parse_qs

class OAuth2Client:
    """
    OAuth 2.0 client implementation.
    Handles authorization code flow.
    """
    
    def __init__(self, client_id: str, client_secret: str, 
                 redirect_uri: str, auth_server: str):
        """
        Initialize OAuth 2.0 client.
        
        Args:
            client_id: Client identifier
            client_secret: Client secret
            redirect_uri: URI to redirect to after authorization
            auth_server: Authorization server URL
        """
        self.client_id = client_id
        self.client_secret = client_secret
        self.redirect_uri = redirect_uri
        self.auth_server = auth_server
        
        # In-memory storage (use database in production)
        self.authorization_codes = {}
        self.access_tokens = {}
    
    def generate_authorization_url(self, scopes: list, state: str) -> str:
        """
        Generate the authorization URL.
        
        Args:
            scopes: List of requested scopes
            state: Random state to prevent CSRF
        
        Returns:
            str: Authorization URL
        """
        params = {
            'response_type': 'code',
            'client_id': self.client_id,
            'redirect_uri': self.redirect_uri,
            'scope': ' '.join(scopes),
            'state': state
        }
        
        url = f"{self.auth_server}/authorize?{urlencode(params)}"
        print(f"✓ Generated authorization URL")
        return url
    
    def exchange_code_for_token(self, code: str) -> Optional[Dict]:
        """
        Exchange authorization code for access token.
        
        Args:
            code: Authorization code from callback
        
        Returns:
            Dict: Token response if successful
        """
        # Validate code (in production, verify with auth server)
        if code not in self.authorization_codes:
            print("✗ Invalid authorization code")
            return None
        
        auth_data = self.authorization_codes[code]
        
        # Generate access token (in production, auth server does this)
        access_token = secrets.token_hex(32)
        refresh_token = secrets.token_hex(32)
        
        token_data = {
            'access_token': access_token,
            'token_type': 'Bearer',
            'expires_in': 3600,
            'refresh_token': refresh_token,
            'scope': auth_data['scope']
        }
        
        # Store token
        self.access_tokens[access_token] = {
            'user_id': auth_data['user_id'],
            'scope': auth_data['scope'],
            'expires_at': time.time() + 3600
        }
        
        # Remove used code
        del self.authorization_codes[code]
        
        print(f"✓ Exchanged code for access token")
        return token_data
    
    def refresh_access_token(self, refresh_token: str) -> Optional[Dict]:
        """
        Refresh access token using refresh token.
        
        Args:
            refresh_token: Refresh token
        
        Returns:
            Dict: New token response if successful
        """
        # In production, verify with auth server
        # For demo, we'll just generate a new token
        
        access_token = secrets.token_hex(32)
        
        token_data = {
            'access_token': access_token,
            'token_type': 'Bearer',
            'expires_in': 3600,
            'refresh_token': refresh_token  # Same refresh token
        }
        
        print(f"✓ Refreshed access token")
        return token_data
    
    def validate_token(self, token: str) -> Optional[Dict]:
        """
        Validate an access token.
        
        Args:
            token: Access token to validate
        
        Returns:
            Dict: Token data if valid
        """
        token_data = self.access_tokens.get(token)
        
        if not token_data:
            print("✗ Invalid token")
            return None
        
        # Check expiration
        if time.time() > token_data['expires_at']:
            print("✗ Token expired")
            del self.access_tokens[token]
            return None
        
        print(f"✓ Token validated for user {token_data['user_id']}")
        return token_data


class OAuth2AuthorizationServer:
    """
    OAuth 2.0 authorization server (simplified).
    In production, this would be a separate service like Auth0, Okta, etc.
    """
    
    def __init__(self):
        self.clients = {
            'demo-app': {
                'client_secret': 'demo-secret',
                'redirect_uris': ['http://localhost:3000/callback']
            }
        }
        self.users = {
            'alice': {'password': 'password123', 'user_id': 1}
        }
        self.authorization_codes = {}
    
    def authorize(self, client_id: str, redirect_uri: str, 
                  username: str, password: str, scope: str) -> Optional[str]:
        """
        Authorize a user and generate authorization code.
        
        Args:
            client_id: Client identifier
            redirect_uri: Redirect URI
            username: Username
            password: Password
            scope: Requested scope
        
        Returns:
            str: Authorization code if successful
        """
        # Validate client
        client = self.clients.get(client_id)
        if not client:
            print("✗ Invalid client")
            return None
        
        if redirect_uri not in client['redirect_uris']:
            print("✗ Invalid redirect URI")
            return None
        
        # Validate user
        user = self.users.get(username)
        if not user or user['password'] != password:
            print("✗ Invalid credentials")
            return None
        
        # Generate authorization code
        code = secrets.token_hex(16)
        self.authorization_codes[code] = {
            'client_id': client_id,
            'user_id': user['user_id'],
            'scope': scope,
            'expires_at': time.time() + 600  # 10 minutes
        }
        
        print(f"✓ Generated authorization code for user {username}")
        return code
    
    def token(self, client_id: str, client_secret: str, 
              code: str) -> Optional[Dict]:
        """
        Exchange authorization code for tokens.
        
        Args:
            client_id: Client identifier
            client_secret: Client secret
            code: Authorization code
        
        Returns:
            Dict: Token response if successful
        """
        # Validate client
        client = self.clients.get(client_id)
        if not client or client['client_secret'] != client_secret:
            print("✗ Invalid client credentials")
            return None
        
        # Validate code
        auth_code = self.authorization_codes.get(code)
        if not auth_code:
            print("✗ Invalid authorization code")
            return None
        
        if time.time() > auth_code['expires_at']:
            print("✗ Authorization code expired")
            del self.authorization_codes[code]
            return None
        
        # Generate tokens
        access_token = secrets.token_hex(32)
        refresh_token = secrets.token_hex(32)
        
        token_data = {
            'access_token': access_token,
            'token_type': 'Bearer',
            'expires_in': 3600,
            'refresh_token': refresh_token,
            'scope': auth_code['scope']
        }
        
        # Remove used code
        del self.authorization_codes[code]
        
        print(f"✓ Issued tokens for client {client_id}")
        return token_data


# Example usage
print("=== OAuth 2.0 Demo ===\n")

# Setup
auth_server = OAuth2AuthorizationServer()
client = OAuth2Client(
    client_id='demo-app',
    client_secret='demo-secret',
    redirect_uri='http://localhost:3000/callback',
    auth_server='https://auth.example.com'
)

# Step 1: User initiates login
print("--- Step 1: Initiate Authorization ---\n")
state = secrets.token_hex(16)
auth_url = client.generate_authorization_url(
    scopes=['profile', 'email'],
    state=state
)
print(f"Authorization URL: {auth_url}\n")

# Step 2: User logs in (simulated)
print("--- Step 2: User Authorizes ---\n")
code = auth_server.authorize(
    client_id='demo-app',
    redirect_uri='http://localhost:3000/callback',
    username='alice',
    password='password123',
    scope='profile email'
)

if code:
    print(f"Authorization Code: {code}\n")
    
    # Step 3: Exchange code for token
    print("--- Step 3: Exchange Code for Token ---\n")
    token_response = client.exchange_code_for_token(code)
    
    if token_response:
        print(f"Token Response: {json.dumps(token_response, indent=2)}\n")
        
        # Step 4: Use access token
        print("--- Step 4: Use Access Token ---\n")
        token_data = client.validate_token(token_response['access_token'])
        
        if token_data:
            print(f"Token validated: {token_data}\n")
        
        # Step 5: Refresh token
        print("--- Step 5: Refresh Access Token ---\n")
        new_token_response = client.refresh_access_token(
            token_response['refresh_token']
        )
        
        if new_token_response:
            print(f"New Token Response: {json.dumps(new_token_response, indent=2)}")
```

**Output:**
```
=== OAuth 2.0 Demo ===

--- Step 1: Initiate Authorization ---

✓ Generated authorization URL
Authorization URL: https://auth.example.com/authorize?response_type=code&client_id=demo-app&redirect_uri=http%3A%2F%2Flocalhost%3A3000%2Fcallback&scope=profile+email&state=abc123...

--- Step 2: User Authorizes ---

✓ Generated authorization code for user alice
Authorization Code: f3e8d7c2a1b4... (16-character code)

--- Step 3: Exchange Code for Token ---

✓ Exchanged code for access token
Token Response: {
  "access_token": "a1b2c3d4e5f6g7h8i9j0k1l2m3n4o5p6...",
  "token_type": "Bearer",
  "expires_in": 3600,
  "refresh_token": "x1y2z3w4v5u6t7s8r9q0p1o2n3m4l5k6...",
  "scope": "profile email"
}

--- Step 4: Use Access Token ---

✓ Token validated for user 1
Token validated: {'user_id': 1, 'scope': 'profile email', 'expires_at': 1709254800}

--- Step 5: Refresh Access Token ---

✓ Refreshed access token
New Token Response: {
  "access_token": "q1w2e3r4t5y6u7i8o9p0a1s2d3f4g5h6...",
  "token_type": "Bearer",
  "expires_in": 3600,
  "refresh_token": "x1y2z3w4v5u6t7s8r9q0p1o2n3m4l5k6..."
}
```

**OAuth 2.0 vs. JWT:**

| Aspect | OAuth 2.0 | JWT |
|--------|-----------|-----|
| **Purpose** | Authorization framework | Token format |
| **What it does** | Delegated authorization | Self-contained tokens |
| **Use case** | "Sign in with Google/Facebook" | API authentication |
| **Token format** | Can use JWT or opaque tokens | Always JWT |
| **State** | Authorization server maintains state | Stateless |
| **Revocation** | Easy (revoke at auth server) | Hard (wait for expiration) |
| **Relationship** | OAuth 2.0 can use JWT tokens | JWT can be used within OAuth 2.0 |

### **OpenID Connect (OIDC)**

OpenID Connect is an identity layer on top of OAuth 2.0. It adds authentication to OAuth 2.0.

**What OIDC Adds:**

```
OAuth 2.0: Authorization
  • "This app can access my data"
  • Doesn't tell you who the user is
  
OpenID Connect: Authentication
  • "This app can access my data"
  • PLUS "Here's who I am"
  • Provides identity information
```

**OIDC Flow:**

```
┌─────────────────────────────────────────────────────────────────┐
│              OPENID CONNECT FLOW                                 │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Same as OAuth 2.0, but returns an ID token along with           │
│  access token:                                                  │
│                                                                 │
│  Token Response:                                                │
│  {                                                              │
│    "access_token": "ey...",      ← Access API resources          │
│    "id_token": "ey...",          ← Contains user identity         │
│    "token_type": "Bearer",                                       │
│    "expires_in": 3600                                           │
│  }                                                              │
│                                                                 │
│  ID Token is a JWT with user claims:                            │
│  {                                                              │
│    "sub": "1234567890",          ← Subject (user ID)             │
│    "name": "Alice Doe",                                           │
│    "email": "alice@example.com",                                 │
│    "email_verified": true,                                       │
│    "iss": "https://auth.example.com",  ← Issuer                  │
│    "aud": "demo-app",              ← Audience                   │
│    "exp": 1709254800,                                           │
│    "iat": 1709251200                                            │
│  }                                                              │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**OIDC Benefits:**

| Benefit | Description |
|---------|-------------|
| **Standardized identity** | Consistent user info across providers |
| **No need to store passwords** | Users authenticate with existing accounts |
| **Reduced development time** | Don't build your own auth system |
| **Security** | Leverages OAuth 2.0 security |
| **Interoperability** | Works with many providers (Google, Facebook, etc.) |

---

## **Authorization: What Can You Do?**

Authentication answers "Who are you?" Authorization answers "What can you do?"

### **Authorization Models**

```
┌─────────────────────────────────────────────────────────────────┐
│                 AUTHORIZATION MODELS                             │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  1. Access Control Lists (ACL)                                  │
│     ────────────────────────────────────────────────────────   │
│     • Direct: "User X can access Resource Y"                    │
│     • Simple, but doesn't scale well                            │
│     • Hard to manage for many users and resources               │
│                                                                 │
│  2. Role-Based Access Control (RBAC)                            │
│     ────────────────────────────────────────────────────────   │
│     • Indirect: "User X has Role R, Role R can do Action A"     │
│     • Scales better than ACLs                                   │
│     • Common in enterprise systems                              │
│                                                                 │
│  3. Attribute-Based Access Control (ABAC)                       │
│     ────────────────────────────────────────────────────────   │
│     • Flexible: "User with attributes can do action based       │
│                on resource attributes"                          │
│     • Most flexible, but most complex                           │
│     • Fine-grained control                                      │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### **RBAC (Role-Based Access Control)**

RBAC assigns permissions to roles, and roles to users.

**RBAC Model:**

```
┌─────────────────────────────────────────────────────────────────┐
│                    RBAC MODEL                                    │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│                  Users                                          │
│            ┌─────┴─────┬─────────────┐                          │
│            ↓           ↓             ↓                          │
│         Alice        Bob         Charlie                        │
│            │           │             │                          │
│            │           │             │                          │
│            ↓           ↓             ↓                          │
│         Roles      Roles        Roles                           │
│       ┌──────┐   ┌──────┐    ┌──────┐                          │
│       │Admin │   │Editor│    │Guest │                          │
│       └──┬───┘   └──┬───┘    └──┬───┘                          │
│          │          │           │                              │
│          └────┬─────┴─────┬─────┘                              │
│               ↓           ↓                                   │
│           Permissions  Permissions                             │
│          ┌────────┐  ┌────────┐                               │
│          │Create  │  │Read    │                               │
│          │Delete  │  │Update  │                               │
│          │Manage  │  │Comment │                               │
│          └────────┘  └────────┘                               │
│                                                                 │
│  Benefits:                                                      │
│  • Scalable: Add users to roles, not individual permissions    │
│  • Manageable: Change role permissions, affects all users      │
│  • Auditable: Easy to see who has what access                   │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**RBAC Example:**

```
Roles and Permissions:

Admin Role:
  - users.create
  - users.read
  - users.update
  - users.delete
  - posts.create
  - posts.read
  - posts.update
  - posts.delete

Editor Role:
  - users.read
  - posts.create
  - posts.read
  - posts.update
  - posts.delete (own posts only)

Guest Role:
  - posts.read
  - comments.create

User-Role Assignments:
  - Alice: Admin
  - Bob: Editor
  - Charlie: Guest
```

**Code Example — RBAC Implementation:**

```python
from typing import Dict, List, Set
from enum import Enum

class Permission(Enum):
    """Permissions in the system."""
    USERS_CREATE = "users.create"
    USERS_READ = "users.read"
    USERS_UPDATE = "users.update"
    USERS_DELETE = "users.delete"
    POSTS_CREATE = "posts.create"
    POSTS_READ = "posts.read"
    POSTS_UPDATE = "posts.update"
    POSTS_DELETE = "posts.delete"
    COMMENTS_CREATE = "comments.create"


class Role:
    """Represents a role with permissions."""
    
    def __init__(self, name: str, permissions: Set[Permission]):
        """
        Initialize role.
        
        Args:
            name: Role name
            permissions: Set of permissions
        """
        self.name = name
        self.permissions = permissions
    
    def has_permission(self, permission: Permission) -> bool:
        """Check if role has a permission."""
        return permission in self.permissions


class User:
    """Represents a user with roles."""
    
    def __init__(self, user_id: int, username: str):
        """
        Initialize user.
        
        Args:
            user_id: User ID
            username: Username
        """
        self.user_id = user_id
        self.username = username
        self.roles: Set[Role] = set()
    
    def add_role(self, role: Role):
        """Add a role to the user."""
        self.roles.add(role)
        print(f"✓ Added role '{role.name}' to user '{self.username}'")
    
    def remove_role(self, role: Role):
        """Remove a role from the user."""
        self.roles.discard(role)
        print(f"✓ Removed role '{role.name}' from user '{self.username}'")
    
    def has_permission(self, permission: Permission) -> bool:
        """
        Check if user has a permission (via any role).
        
        Args:
            permission: Permission to check
        
        Returns:
            bool: True if user has permission
        """
        for role in self.roles:
            if role.has_permission(permission):
                return True
        return False
    
    def get_permissions(self) -> Set[Permission]:
        """Get all permissions from all roles."""
        permissions = set()
        for role in self.roles:
            permissions.update(role.permissions)
        return permissions


class RBACSystem:
    """
    Role-Based Access Control system.
    """
    
    def __init__(self):
        """Initialize RBAC system."""
        # Define roles
        self.roles: Dict[str, Role] = {
            'admin': Role('Admin', {
                Permission.USERS_CREATE,
                Permission.USERS_READ,
                Permission.USERS_UPDATE,
                Permission.USERS_DELETE,
                Permission.POSTS_CREATE,
                Permission.POSTS_READ,
                Permission.POSTS_UPDATE,
                Permission.POSTS_DELETE,
            }),
            'editor': Role('Editor', {
                Permission.USERS_READ,
                Permission.POSTS_CREATE,
                Permission.POSTS_READ,
                Permission.POSTS_UPDATE,
                Permission.POSTS_DELETE,
            }),
            'guest': Role('Guest', {
                Permission.POSTS_READ,
                Permission.COMMENTS_CREATE,
            })
        }
        
        # Users
        self.users: Dict[int, User] = {}
    
    def create_user(self, user_id: int, username: str) -> User:
        """
        Create a new user.
        
        Args:
            user_id: User ID
            username: Username
        
        Returns:
            User: Created user
        """
        user = User(user_id, username)
        self.users[user_id] = user
        print(f"✓ Created user '{username}' (ID: {user_id})")
        return user
    
    def assign_role(self, user_id: int, role_name: str) -> bool:
        """
        Assign a role to a user.
        
        Args:
            user_id: User ID
            role_name: Role name
        
        Returns:
            bool: True if successful
        """
        user = self.users.get(user_id)
        role = self.roles.get(role_name)
        
        if not user:
            print(f"✗ User not found: {user_id}")
            return False
        
        if not role:
            print(f"✗ Role not found: {role_name}")
            return False
        
        user.add_role(role)
        return True
    
    def check_permission(self, user_id: int, permission: Permission) -> bool:
        """
        Check if a user has a permission.
        
        Args:
            user_id: User ID
            permission: Permission to check
        
        Returns:
            bool: True if user has permission
        """
        user = self.users.get(user_id)
        
        if not user:
            print(f"✗ User not found: {user_id}")
            return False
        
        has_perm = user.has_permission(permission)
        user_name = user.username
        perm_str = permission.value
        
        status = "✓" if has_perm else "✗"
        print(f"{status} User '{user_name}' {('has' if has_perm else 'does NOT have')} permission '{perm_str}'")
        
        return has_perm


# Example usage
print("=== RBAC Demo ===\n")

rbac = RBACSystem()

# Create users
print("--- Creating Users ---\n")
alice = rbac.create_user(1, 'Alice')
bob = rbac.create_user(2, 'Bob')
charlie = rbac.create_user(3, 'Charlie')
print()

# Assign roles
print("--- Assigning Roles ---\n")
rbac.assign_role(1, 'admin')
rbac.assign_role(2, 'editor')
rbac.assign_role(3, 'guest')
print()

# Check permissions
print("--- Checking Permissions ---\n")

# Alice (Admin) should have all permissions
print("Alice (Admin):")
rbac.check_permission(1, Permission.USERS_CREATE)
rbac.check_permission(1, Permission.POSTS_READ)
print()

# Bob (Editor) should have limited permissions
print("Bob (Editor):")
rbac.check_permission(2, Permission.USERS_CREATE)  # Should fail
rbac.check_permission(2, Permission.POSTS_CREATE)  # Should succeed
print()

# Charlie (Guest) should have minimal permissions
print("Charlie (Guest):")
rbac.check_permission(3, Permission.USERS_READ)    # Should fail
rbac.check_permission(3, Permission.POSTS_READ)    # Should succeed
rbac.check_permission(3, Permission.COMMENTS_CREATE)  # Should succeed
print()

# Show all permissions for each user
print("--- User Permissions Summary ---\n")
for user in rbac.users.values():
    perms = user.get_permissions()
    role_names = [r.name for r in user.roles]
    print(f"{user.username} (Roles: {', '.join(role_names)}):")
    for perm in sorted(perms, key=lambda p: p.value):
        print(f"  - {perm.value}")
    print()
```

**Output:**
```
=== RBAC Demo ===

--- Creating Users ---

✓ Created user 'Alice' (ID: 1)
✓ Created user 'Bob' (ID: 2)
✓ Created user 'Charlie' (ID: 3)

--- Assigning Roles ---

✓ Added role 'Admin' to user 'Alice'
✓ Added role 'Editor' to user 'Bob'
✓ Added role 'Guest' to user 'Charlie'

--- Checking Permissions ---

Alice (Admin):
✓ User 'Alice' has permission 'users.create'
✓ User 'Alice' has permission 'posts.read'

Bob (Editor):
✗ User 'Bob' does NOT have permission 'users.create'
✓ User 'Bob' has permission 'posts.create'

Charlie (Guest):
✗ User 'Charlie' does NOT have permission 'users.read'
✓ User 'Charlie' has permission 'posts.read'
✓ User 'Charlie' has permission 'comments.create'

--- User Permissions Summary ---

Alice (Roles: Admin):
  - posts.create
  - posts.delete
  - posts.read
  - posts.update
  - users.create
  - users.delete
  - users.read
  - users.update

Bob (Roles: Editor):
  - comments.create
  - posts.create
  - posts.delete
  - posts.read
  - posts.update
  - users.read

Charlie (Roles: Guest):
  - comments.create
  - posts.read
```

**RBAC Best Practices:**

| Practice | Description |
|----------|-------------|
| **Principle of Least Privilege** | Users only get minimum required permissions |
| **Role Hierarchy** | Use inheritance to avoid duplication |
| **Regular Audits** | Review role assignments and permissions |
| **Separation of Duties** | Don't give one person too much power |
| **Dynamic Roles** | Consider context (time, location) for additional security |

### **ABAC (Attribute-Based Access Control)**

ABAC is more flexible than RBAC, using attributes to determine access.

**ABAC Model:**

```
┌─────────────────────────────────────────────────────────────────┐
│                    ABAC MODEL                                    │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Access Decision = F(User, Resource, Action, Environment)      │
│                                                                 │
│  User Attributes:                                              │
│  • user_id: 123                                                │
│  • department: finance                                         │
│  • level: senior                                               │
│  • clearance: secret                                           │
│                                                                 │
│  Resource Attributes:                                           │
│  • resource_id: doc-456                                        │
│  • type: financial_report                                      │
│  • classification: confidential                                │
│  • owner: alice                                                │
│                                                                 │
│  Action Attributes:                                             │
│  • action: read                                                │
│  • action: write                                               │
│  • action: delete                                              │
│                                                                 │
│  Environment Attributes:                                        │
│  • time: 9am-5pm                                               │
│  • location: office                                            │
│  • network: secure                                             │
│                                                                 │
│  Policy:                                                       │
│  IF user.department == resource.owner                          │
│     AND action == read                                         │
│     AND resource.classification <= user.clearance              │
│  THEN ALLOW                                                    │
│  ELSE DENY                                                     │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**ABAC vs RBAC:**

| Aspect | RBAC | ABAC |
|--------|------|------|
| **Flexibility** | Medium (fixed roles) | High (dynamic attributes) |
| **Complexity** | Low | High |
| **Scalability** | Good (roles scale) | Better (attributes scale) |
| **Management** | Role-based | Policy-based |
| **Granularity** | Coarse | Fine |
| **Performance** | Fast (simple lookups) | Slower (policy evaluation) |
| **Best For** | Static, structured permissions | Dynamic, context-aware permissions |

**Code Example — ABAC Implementation:**

```python
from typing import Dict, Any, List
from enum import Enum
from datetime import time

class Decision(Enum):
    """Access control decision."""
    ALLOW = "allow"
    DENY = "deny"

class AttributeType(Enum):
    """Types of attributes."""
    USER = "user"
    RESOURCE = "resource"
    ACTION = "action"
    ENVIRONMENT = "environment"

class Policy:
    """
    An ABAC policy that defines access rules.
    """
    
    def __init__(self, name: str, description: str):
        """
        Initialize policy.
        
        Args:
            name: Policy name
            description: Policy description
        """
        self.name = name
        self.description = description
        self.rules: List[Dict] = []
    
    def add_rule(self, conditions: Dict, decision: Decision):
        """
        Add a rule to the policy.
        
        Args:
            conditions: Dictionary of attribute conditions
            decision: ALLOW or DENY
        """
        rule = {
            'conditions': conditions,
            'decision': decision
        }
        self.rules.append(rule)
        print(f"✓ Added rule to policy '{self.name}': {decision.value}")
    
    def evaluate(self, user_attrs: Dict, resource_attrs: Dict, 
                 action_attrs: Dict, env_attrs: Dict) -> Decision:
        """
        Evaluate policy against attributes.
        
        Args:
            user_attrs: User attributes
            resource_attrs: Resource attributes
            action_attrs: Action attributes
            env_attrs: Environment attributes
        
        Returns:
            Decision: ALLOW or DENY
        """
        # Default to DENY
        final_decision = Decision.DENY
        
        for rule in self.rules:
            conditions = rule['conditions']
            decision = rule['decision']
            
            # Check if all conditions match
            matches = True
            
            for attr_path, expected_value in conditions.items():
                parts = attr_path.split('.')
                
                # Navigate to the attribute
                if parts[0] == 'user':
                    attrs = user_attrs
                elif parts[0] == 'resource':
                    attrs = resource_attrs
                elif parts[0] == 'action':
                    attrs = action_attrs
                elif parts[0] == 'environment':
                    attrs = env_attrs
                else:
                    matches = False
                    break
                
                # Get nested value
                value = attrs
                for part in parts[1:]:
                    if isinstance(value, dict) and part in value:
                        value = value[part]
                    else:
                        matches = False
                        break
                
                # Check condition
                if not matches:
                    break
                
                if callable(expected_value):
                    # Custom condition function
                    if not expected_value(value):
                        matches = False
                        break
                elif value != expected_value:
                    matches = False
                    break
            
            if matches:
                final_decision = decision
                break
        
        return final_decision


class ABACSystem:
    """
    Attribute-Based Access Control system.
    """
    
    def __init__(self):
        """Initialize ABAC system."""
        self.policies: Dict[str, Policy] = {}
        self._setup_default_policies()
    
    def _setup_default_policies(self):
        """Setup default policies."""
        # Policy: Users can read their own resources
        own_resources = Policy(
            "own_resources",
            "Users can read and update their own resources"
        )
        own_resources.add_rule(
            conditions={
                'user.user_id': lambda v: v is not None,
                'resource.owner_id': lambda v: v is not None,
                'action.type': lambda v: v in ['read', 'update']
            },
            decision=Decision.ALLOW
        )
        
        # Condition: user must own resource
        # This is simplified - in real system, would compare user_id == owner_id
        self.policies['own_resources'] = own_resources
        
        # Policy: Admins can do anything
        admin_access = Policy(
            "admin_access",
            "Admins have full access"
        )
        admin_access.add_rule(
            conditions={
                'user.role': 'admin'
            },
            decision=Decision.ALLOW
        )
        self.policies['admin_access'] = admin_access
    
    def add_policy(self, policy: Policy):
        """
        Add a policy to the system.
        
        Args:
            policy: Policy to add
        """
        self.policies[policy.name] = policy
    
    def check_access(self, user: Dict, resource: Dict, 
                     action: Dict, environment: Dict) -> Decision:
        """
        Check if access is allowed.
        
        Args:
            user: User attributes
            resource: Resource attributes
            action: Action attributes
            environment: Environment attributes
        
        Returns:
            Decision: ALLOW or DENY
        """
        # Evaluate all policies
        for policy_name, policy in self.policies.items():
            decision = policy.evaluate(user, resource, action, environment)
            
            if decision == Decision.ALLOW:
                print(f"✓ Policy '{policy_name}' allows access")
                return Decision.ALLOW
        
        print("✗ No policy allows access")
        return Decision.DENY
    
    def check_access_simple(self, user_id: int, resource_id: int, 
                           action: str, is_admin: bool = False) -> bool:
        """
        Simplified access check.
        
        Args:
            user_id: User ID
            resource_id: Resource ID
            action: Action (read, update, delete)
            is_admin: Whether user is admin
        
        Returns:
            bool: True if access allowed
        """
        user = {'user_id': user_id, 'role': 'admin' if is_admin else 'user'}
        resource = {'resource_id': resource_id, 'owner_id': resource_id}  # Simplified
        action_attrs = {'type': action}
        environment = {}
        
        decision = self.check_access(user, resource, action_attrs, environment)
        return decision == Decision.ALLOW


# Example usage
print("=== ABAC Demo ===\n")

abac = ABACSystem()

# Check access for regular user
print("--- Regular User Access ---\n")
print("User 123 trying to read resource 456:")
result = abac.check_access_simple(123, 456, 'read', is_admin=False)
print(f"Result: {'ALLOWED' if result else 'DENIED'}\n")

# Check access for admin
print("--- Admin Access ---\n")
print("Admin trying to delete resource 789:")
result = abac.check_access_simple(999, 789, 'delete', is_admin=True)
print(f"Result: {'ALLOWED' if result else 'DENIED'}\n")

# Add custom policy
print("--- Custom Policy ---\n")
finance_policy = Policy(
    "finance_only",
    "Only finance department can access financial reports"
)
finance_policy.add_rule(
    conditions={
        'user.department': 'finance',
        'resource.type': 'financial_report',
        'action.type': 'read'
    },
    decision=Decision.ALLOW
)
abac.add_policy(finance_policy)

# Check with custom policy
user = {'user_id': 1, 'role': 'user', 'department': 'finance'}
resource = {'resource_id': 100, 'type': 'financial_report'}
action = {'type': 'read'}
environment = {}

print("Finance user trying to read financial report:")
result = abac.check_access(user, resource, action, environment)
print(f"Result: {result.value}\n")

print("Engineering user trying to read financial report:")
user['department'] = 'engineering'
result = abac.check_access(user, resource, action, environment)
print(f"Result: {result.value}")
```

**Output:**
```
=== ABAC Demo ===

--- Regular User Access ---

User 123 trying to read resource 456:
✓ Policy 'own_resources' allows access
Result: ALLOWED

--- Admin Access ---

Admin trying to delete resource 789:
✓ Policy 'admin_access' allows access
Result: ALLOWED

--- Custom Policy ---

✓ Added rule to policy 'finance_only': allow
✓ Added rule to policy 'finance_only': allow
✓ Added rule to policy 'finance_only': allow

Finance user trying to read financial report:
✓ Policy 'finance_only' allows access
Result: allow

Engineering user trying to read financial report:
✗ No policy allows access
Result: deny
```

---

## **API Security: Protecting Your Endpoints**

APIs are the gateway to your system. Securing them is critical.

### **API Security Best Practices**

```
┌─────────────────────────────────────────────────────────────────┐
│              API SECURITY CHECKLIST                              │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Authentication & Authorization                                  │
│  □ Use HTTPS only (TLS 1.2+)                                    │
│  □ Implement proper authentication (JWT, OAuth)                  │
│  □ Validate tokens on every request                              │
│  □ Implement rate limiting                                      │
│  □ Use API keys for service-to-service communication            │
│                                                                 │
│  Input Validation                                               │
│  □ Validate all input data                                      │
│  □ Use parameterized queries to prevent SQL injection           │
│  □ Sanitize output to prevent XSS                                │
│  □ Use content type validation                                  │
│                                                                 │
│  Access Control                                                 │
│  □ Implement proper authorization (RBAC/ABAC)                   │
│  □ Use principle of least privilege                              │
│  □ Regularly audit permissions                                  │
│                                                                 │
│  Data Protection                                                │
│  □ Encrypt sensitive data at rest                               │
│  □ Encrypt sensitive data in transit                            │
│  □ Never log sensitive data (passwords, tokens)                 │
│  □ Use secure headers (CSP, HSTS, X-Frame-Options)             │
│                                                                 │
│  Error Handling                                                 │
│  □ Return generic error messages to clients                     │
│  □ Log detailed errors server-side                              │
│  □ Don't expose stack traces                                    │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### **Rate Limiting**

Rate limiting protects your API from abuse and ensures fair usage.

**Why Rate Limiting?**

```
Without Rate Limiting:
  • One user can make 1,000,000 requests/second
  • API becomes unresponsive for everyone
  • Database overloaded
  • Costs spike
  • DDoS attacks easier

With Rate Limiting:
  • Each user limited to 100 requests/minute
  • API remains responsive
  • Costs predictable
  • DDoS attacks mitigated
  • Fair usage for all
```

**Rate Limiting Strategies:**

| Strategy | Description | Use Case |
|----------|-------------|----------|
| **Fixed Window** | Reset counter at fixed interval | Simple, but has burst issues |
| **Sliding Window** | Rolling time window | More accurate, more complex |
| **Token Bucket** | Tokens refill at rate, requests consume tokens | Burst-friendly |
| **Leaky Bucket** | Requests fill bucket, leak out at rate | Smooths traffic |

**Code Example — Token Bucket Rate Limiter:**

```python
import time
from typing import Dict
from collections import defaultdict

class TokenBucketRateLimiter:
    """
    Token bucket rate limiter.
    Allows burst traffic while enforcing long-term rate limits.
    """
    
    def __init__(self, capacity: int, refill_rate: float):
        """
        Initialize rate limiter.
        
        Args:
            capacity: Maximum tokens (burst size)
            refill_rate: Tokens per second refill rate
        """
        self.capacity = capacity
        self.refill_rate = refill_rate
        
        # Bucket state: {identifier: (tokens, last_refill_time)}
        self.buckets: Dict[str, tuple] = defaultdict(lambda: (capacity, time.time()))
    
    def _refill(self, identifier: str, now: float):
        """Refill tokens for a given identifier."""
        tokens, last_refill = self.buckets[identifier]
        
        # Calculate time elapsed
        elapsed = now - last_refill
        
        # Refill tokens
        new_tokens = min(self.capacity, tokens + elapsed * self.refill_rate)
        
        # Update bucket
        self.buckets[identifier] = (new_tokens, now)
    
    def allow_request(self, identifier: str) -> bool:
        """
        Check if a request is allowed for the given identifier.
        
        Args:
            identifier: Unique identifier (user ID, API key, IP, etc.)
        
        Returns:
            bool: True if request allowed, False otherwise
        """
        now = time.time()
        
        # Refill tokens
        self._refill(identifier, now)
        
        # Get current tokens
        tokens, _ = self.buckets[identifier]
        
        if tokens >= 1:
            # Consume one token
            self.buckets[identifier] = (tokens - 1, now)
            return True
        
        return False
    
    def get_remaining_tokens(self, identifier: str) -> float:
        """
        Get remaining tokens for an identifier.
        
        Args:
            identifier: Unique identifier
        
        Returns:
            float: Remaining tokens
        """
        now = time.time()
        self._refill(identifier, now)
        tokens, _ = self.buckets[identifier]
        return tokens


# Example usage
print("=== Token Bucket Rate Limiter Demo ===\n")

# Create rate limiter: 10 tokens capacity, 2 tokens/second refill
limiter = TokenBucketRateLimiter(capacity=10, refill_rate=2)

# Simulate requests from user "user123"
user_id = "user123"
print(f"Rate limit: {limiter.capacity} tokens, {limiter.refill_rate} tokens/sec refill\n")

print("--- Sending Requests ---\n")

# Send burst of requests (more than capacity)
for i in range(15):
    allowed = limiter.allow_request(user_id)
    remaining = limiter.get_remaining_tokens(user_id)
    status = "✓ ALLOWED" if allowed else "✗ RATE LIMITED"
    print(f"Request {i+1}: {status} (remaining: {remaining:.1f} tokens)")
    
    if i == 9:  # After 10 requests, wait a bit
        print("\n⏳ Waiting 3 seconds for refill...\n")
        time.sleep(3)

print(f"\nFinal remaining tokens: {limiter.get_remaining_tokens(user_id):.1f}")

# Show how refill works
print("\n--- Token Refill Over Time ---\n")
limiter = TokenBucketRateLimiter(capacity=5, refill_rate=1)
user_id = "user456"

# Use all tokens
print("Using all tokens...")
for i in range(5):
    limiter.allow_request(user_id)
print(f"Tokens after using all: {limiter.get_remaining_tokens(user_id):.1f}\n")

# Wait and show refill
for seconds in [1, 2, 3, 4, 5]:
    time.sleep(1)
    remaining = limiter.get_remaining_tokens(user_id)
    print(f"After {seconds} second(s): {remaining:.1f} tokens")
```

**Output:**
```
=== Token Bucket Rate Limiter Demo ===

Rate limit: 10 tokens, 2 tokens/sec refill

--- Sending Requests ---

Request 1: ✓ ALLOWED (remaining: 9.0 tokens)
Request 2: ✓ ALLOWED (remaining: 8.0 tokens)
Request 3: ✓ ALLOWED (remaining: 7.0 tokens)
Request 4: ✓ ALLOWED (remaining: 6.0 Keys)
Request 5: ✓ ALLOWED (remaining: 5.0 tokens)
Request 6: ✓ ALLOWED (remaining: 4.0 tokens)
Request 7: ✓ ALLOWED (remaining: 3.0 tokens)
Request 8: ✓ ALLOWED (remaining: 2.0 tokens)
Request 9: ✓ ALLOWED (remaining: 1.0 tokens)
Request 10: ✓ ALLOWED (remaining: 0.0 tokens)

⏳ Waiting 3 seconds for refill...

Request 11: ✓ ALLOWED (remaining: 4.0 tokens)
Request 12: ✓ ALLOWED (remaining: 3.0 tokens)
Request 13: ✓ ALLOWED (remaining: 2.0 tokens)
Request 14: ✓ ALLOWED (remaining: 1.0 tokens)
Request 15: ✓ ALLOWED (remaining: 0.0 tokens)

Final remaining tokens: 0.0

--- Token Refill Over Time ---

Using all tokens...
Tokens after using all: 0.0

After 1 second(s): 1.0 tokens
After 2 second(s): 2.0 tokens
After 3 second(s): 3.0 tokens
After 4 second(s): 4.0 tokens
After 5 second(s): 5.0 tokens
```

**Rate Limiting by Use Case:**

| Use Case | Rate Limit | Reasoning |
|----------|------------|-----------|
| Public API | 1000 requests/hour | Prevent abuse, ensure fairness |
| Authenticated API | 10,000 requests/hour | Higher limit for trusted users |
| Internal API | 100,000 requests/hour | Higher for internal services |
| Write operations | 100 requests/minute | Limit database writes |
| Critical operations | 10 requests/minute | Strict limit for expensive ops |

### **API Keys**

API keys identify the calling application or service.

**API Key Best Practices:**

```
✓ DO:
  • Use long, random keys (at least 32 characters)
  • Store keys securely (environment variables, secret managers)
  • Rotate keys regularly
  • Associate keys with specific permissions (scopes)
  • Monitor key usage
  • Revoke compromised keys immediately

✗ DON'T:
  • Hardcode keys in source code
  • Commit keys to version control
  • Share keys via email/chat
  • Use the same key for all environments
  • Log keys in plain text
  • Use weak keys (simple strings)
```

**Code Example — API Key Management:**

```python
import secrets
import hashlib
import hmac
import base64
import time
from typing import Dict, Optional, Tuple
from enum import Enum

class APIKeyScope(Enum):
    """API key scopes (permissions)."""
    READ = "read"
    WRITE = "write"
    ADMIN = "admin"

class APIKey:
    """Represents an API key with metadata."""
    
    def __init__(self, key_id: str, secret_key: str, scopes: set, 
                 created_by: str, expires_at: Optional[float] = None):
        """
        Initialize API key.
        
        Args:
            key_id: Unique key identifier
            secret_key: Secret key value
            scopes: Set of scopes (permissions)
            created_by: Who created the key
            expires_at: Optional expiration timestamp
        """
        self.key_id = key_id
        self.secret_key = secret_key
        self.scopes = scopes
        self.created_by = created_by
        self.created_at = time.time()
        self.expires_at = expires_at
        self.last_used = None
        self.is_active = True
    
    def has_scope(self, scope: APIKeyScope) -> bool:
        """Check if key has a specific scope."""
        return scope in self.scopes
    
    def is_expired(self) -> bool:
        """Check if key is expired."""
        if self.expires_at is None:
            return False
        return time.time() > self.expires_at
    
    def revoke(self):
        """Revoke the API key."""
        self.is_active = False
    
    def use(self):
        """Record that the key was used."""
        self.last_used = time.time()


class APIKeyManager:
    """
    Manages API keys for authentication and authorization.
    """
    
    def __init__(self):
        """Initialize API key manager."""
        # In production, use a database
        self.keys: Dict[str, APIKey] = {}
        self.key_id_counter = 0
    
    def _generate_key_id(self) -> str:
        """Generate a unique key ID."""
        self.key_id_counter += 1
        return f"key_{self.key_id_counter:06d}"
    
    def _generate_secret_key(self) -> str:
        """Generate a secure secret key."""
        return secrets.token_urlsafe(32)
    
    def create_key(self, name: str, scopes: set, created_by: str, 
                  expires_in_days: Optional[int] = None) -> Tuple[str, str]:
        """
        Create a new API key.
        
        Args:
            name: Key name/description
            scopes: Set of scopes
            created_by: Who is creating the key
            expires_in_days: Optional expiration in days
        
        Returns:
            Tuple[str, str]: (key_id, secret_key)
        
        NOTE: Only return secret_key once! Store it securely.
        """
        key_id = self._generate_key_id()
        secret_key = self._generate_secret_key()
        
        expires_at = None
        if expires_in_days:
            expires_at = time.time() + (expires_in_days * 86400)
        
        api_key = APIKey(
            key_id=key_id,
            secret_key=secret_key,
            scopes=scopes,
            created_by=created_by,
            expires_at=expires_at
        )
        
        self.keys[key_id] = api_key
        
        print(f"✓ Created API key '{name}' (ID: {key_id})")
        print(f"  Scopes: {[s.value for s in scopes]}")
        if expires_at:
            print(f"  Expires: {time.strftime('%Y-%m-%d', time.localtime(expires_at))}")
        
        return key_id, secret_key
    
    def verify_key(self, key_id: str, secret_key: str) -> Optional[APIKey]:
        """
        Verify an API key.
        
        Args:
            key_id: Key ID
            secret_key: Secret key to verify
        
        Returns:
            APIKey if valid, None otherwise
        """
        api_key = self.keys.get(key_id)
        
        if not api_key:
            print(f"✗ API key not found: {key_id}")
            return None
        
        if not api_key.is_active:
            print(f"✗ API key revoked: {key_id}")
            return None
        
        if api_key.is_expired():
            print(f"✗ API key expired: {key_id}")
            return None
        
        # Verify secret key (using constant-time comparison)
        if not hmac.compare_digest(api_key.secret_key.encode(), 
                                    secret_key.encode()):
            print(f"✗ Invalid secret key for: {key_id}")
            return None
        
        # Record usage
        api_key.use()
        print(f"✓ API key verified: {key_id}")
        
        return api_key
    
    def check_scope(self, key_id: str, secret_key: str, 
                   required_scope: APIKeyScope) -> bool:
        """
        Check if an API key has a required scope.
        
        Args:
            key_id: Key ID
            secret_key: Secret key
            required_scope: Required scope
        
        Returns:
            bool: True if key has scope
        """
        api_key = self.verify_key(key_id, secret_key)
        
        if not api_key:
            return False
        
        has_scope = api_key.has_scope(required_scope)
        
        status = "✓" if has_scope else "✗"
        print(f"{status} Key {key_id} {('has' if has_scope else 'does NOT have')} scope '{required_scope.value}'")
        
        return has_scope
    
    def revoke_key(self, key_id: str) -> bool:
        """
        Revoke an API key.
        
        Args:
            key_id: Key ID to revoke
        
        Returns:
            bool: True if revoked
        """
        api_key = self.keys.get(key_id)
        
        if not api_key:
            print(f"✗ API key not found: {key_id}")
            return False
        
        api_key.revoke()
        print(f"✓ Revoked API key: {key_id}")
        return True
    
    def list_keys(self) -> list:
        """List all API keys (without secrets)."""
        key_info = []
        
        for key_id, api_key in self.keys.items():
            info = {
                'key_id': key_id,
                'scopes': [s.value for s in api_key.scopes],
                'created_by': api_key.created_by,
                'created_at': time.strftime('%Y-%m-%d', time.localtime(api_key.created_at)),
                'expires_at': time.strftime('%Y-%m-%d', time.localtime(api_key.expires_at)) if api_key.expires_at else None,
                'last_used': time.strftime('%Y-%m-%d', time.localtime(api_key.last_used)) if api_key.last_used else None,
                'is_active': api_key.is_active
            }
            key_info.append(info)
        
        return key_info


# Example usage
print("=== API Key Management Demo ===\n")

manager = APIKeyManager()

# Create keys with different scopes
print("--- Creating API Keys ---\n")
key1_id, key1_secret = manager.create_key(
    name="Read-only key",
    scopes={APIKeyScope.READ},
    created_by="admin",
    expires_in_days=30
)
print(f"  Key ID: {key1_id}")
print(f"  Secret: {key1_secret}\n")

key2_id, key2_secret = manager.create_key(
    name="Full access key",
    scopes={APIKeyScope.READ, APIKeyScope.WRITE, APIKeyScope.ADMIN},
    created_by="admin"
)
print(f"  Key ID: {key2_id}")
print(f"  Secret: {key2_secret}\n")

# Verify keys
print("--- Verifying API Keys ---\n")
api_key = manager.verify_key(key1_id, key1_secret)
if api_key:
    print(f"  Valid key with scopes: {[s.value for s in api_key.scopes]}\n")

# Check scopes
print("--- Checking Scopes ---\n")
print("Key 1 (read-only):")
manager.check_scope(key1_id, key1_secret, APIKeyScope.READ)  # Should succeed
manager.check_scope(key1_id, key1_secret, APIKeyScope.WRITE)  # Should fail

print("\nKey 2 (full access):")
manager.check_scope(key2_id, key2_secret, APIKeyScope.READ)  # Should succeed
manager.check_scope(key2_id, key2_secret, APIKeyScope.WRITE)  # Should succeed
manager.check_scope(key2_id, key2_secret, APIKeyScope.ADMIN)  # Should succeed

# List all keys
print("\n--- All Keys ---\n")
for key_info in manager.list_keys():
    print(f"Key ID: {key_info['key_id']}")
    print(f"  Scopes: {', '.join(key_info['scopes'])}")
    print(f"  Created: {key_info['created_at']} by {key_info['created_by']}")
    print(f"  Expires: {key_info['expires_at'] or 'Never'}")
    print(f"  Last Used: {key_info['last_used'] or 'Never'}")
    print(f"  Active: {'Yes' if key_info['is_active'] else 'No'}")
    print()

# Revoke a key
print("--- Revoking Key ---\n")
manager.revoke_key(key1_id)

# Try to use revoked key
print("\n--- Trying to Use Revoked Key ---\n")
api_key = manager.verify_key(key1_id, key1_secret)
print(f"Result: {api_key}")
```

**Output:**
```
=== API Key Management Demo ===

--- Creating API Keys ---

✓ Created API key 'Read-only key' (ID: key_000001)
  Scopes: ['read']
  Expires: 2024-03-02
  Key ID: key_000001
  Secret: 8zK7mN3pQ9wR2tY5u...

✓ Created API key 'Full access key' (ID: key_000002)
  Scopes: ['read', 'write', 'admin']
  Key ID: key_000002
  Secret: aB3cD5eF7gH9jK1mN...

--- Verifying API Keys ---

✓ API key verified: key_000001
  Valid key with scopes: ['read']

--- Checking Scopes ---

Key 1 (read-only):
✓ API key verified: key_000001
✓ Key key_000001 has scope 'read'
✓ API key verified: key_000001
✗ Key key_000001 does NOT have scope 'write'

Key 2 (full access):
✓ API key verified: key_000002
✓ Key key_000002 has scope 'read'
✓ API key verified: key_000002
✓ Key key_000002 has scope 'write'
✓ API key verified: key_000002
✓ Key key_000002 has scope 'admin'

--- All Keys ---

Key ID: key_000001
  Scopes: read
  Created: 2024-02-01 by admin
  Expires: 2024-03-02
  Last Used: 2024-02-01
  Active: Yes

Key ID: key_000002
  Scopes: read, write, admin
  Created: 2024-02-01 by admin
  Expires: Never
  Last Used: 2024-02-01
  Active: Yes

--- Revoking Key ---

✓ Revoked API key: key_000001

--- Trying to Use Revoked Key ---

✗ API key revoked: key_000001
Result: None
```

---

## **Data Protection: Encryption at Rest and in Transit**

Protecting data is fundamental to security.

### **Encryption in Transit**

Encryption in transit protects data as it moves across networks.

```
┌─────────────────────────────────────────────────────────────────┐
│              ENCRYPTION IN TRANSIT                               │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Without Encryption:                                            │
│  Client ───plaintext───> Network ───plaintext───> Server        │
│           Anyone can read!                                      │
│                                                                 │
│  With Encryption (TLS/HTTPS):                                   │
│  Client ──TLS handshake──> Network ──TLS handshake──> Server    │
│           │ establishes       │ encrypted data      │            │
│           │ secure channel     │ (unreadable)      │            │
│           ↓                    ↓                    ↓            │
│         Secure              Encrypted            Decrypted      │
│         Channel              Data                Data           │
│                                                                 │
│  TLS Protocol Stack:                                            │
│  ┌─────────────────────────────────────────┐                    │
│  │  Application Layer (HTTP)               │                    │
│  ├─────────────────────────────────────────┤                    │
│  │  TLS/SSL Layer (Encryption)             │                    │
│  │  • Handshake (authentication)           │                    │
│  │  • Key exchange (encryption keys)       │                    │
│  │  • Data encryption (AES, etc.)          │                    │
│  ├─────────────────────────────────────────┤                    │
│  │  Transport Layer (TCP)                  │                    │
│  ├─────────────────────────────────────────┤                    │
│  │  Network Layer (IP)                     │                    │
│  └─────────────────────────────────────────┘                    │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**TLS Best Practices:**

| Practice | Description |
|----------|-------------|
| **Use TLS 1.2+** | Disable TLS 1.0 and 1.1 (insecure) |
| **Strong Ciphers** | Use AES-256, ChaCha20 |
| **Perfect Forward Secrecy** | Use Ephemeral keys |
| **Valid Certificates** | Use trusted CAs, proper expiration |
| **Certificate Pinning** | Prevent MITM attacks (mobile apps) |
| **HSTS** | Force HTTPS connections |
| **OCSP Stapling** | Improve certificate validation |

### **Encryption at Rest**

Encryption at rest protects data stored on disk.

```
┌─────────────────────────────────────────────────────────────────┐
│              ENCRYPTION AT REST                                 │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Without Encryption:                                            │
│  ┌──────────────┐                                              │
│  │   Database   │ ← Anyone with disk access can read data       │
│  │   File       │                                              │
│  └──────────────┘                                              │
│                                                                 │
│  With Encryption:                                              │
│  ┌──────────────┐                                              │
│  │   Encrypted  │ ← Data encrypted with key                    │
│  │   Data       │                                              │
│  └──────┬───────┘                                              │
│         │                                                      │
│         │ Encryption Key                                       │
│         ↓                                                      │
│  ┌──────────────┐                                              │
│  │   Key        │ ← Store keys securely (KMS, HSM)             │
│  │   Management │                                              │
│  └──────────────┘                                              │
│                                                                 │
│  Encryption Algorithms:                                         │
│  • AES-256 (Advanced Encryption Standard)                       │
│    - Industry standard                                          │
│    - Symmetric encryption (same key for encrypt/decrypt)        │
│    - Fast and secure                                            │
│                                                                 │
│  • RSA-4096 (Rivest-Shamir-Adleman)                            │
│    - Asymmetric encryption (public/private key)                 │
│    - Used for key exchange                                      │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**Code Example — Data Encryption:**

```python
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
import base64
import os
from typing import Tuple

class DataEncryptor:
    """
    Encrypts and decrypts data using AES-256.
    """
    
    def __init__(self, password: bytes, salt: bytes = None):
        """
        Initialize encryptor with password-based key derivation.
        
        Args:
            password: Password for key derivation
            salt: Optional salt (use same salt for decrypt)
        """
        if salt is None:
            salt = os.urandom(16)
        
        # Derive key from password
        kdf = PBKDF2HMAC(
            algorithm=hashes.SHA256(),
            length=32,
            salt=salt,
            iterations=100000,
        )
        
        key = base64.urlsafe_b64encode(kdf.derive(password))
        self.cipher = Fernet(key)
        self.salt = salt
    
    def encrypt(self, data: bytes) -> bytes:
        """
        Encrypt data.
        
        Args:
            data: Data to encrypt
        
        Returns:
            bytes: Encrypted data
        """
        encrypted = self.cipher.encrypt(data)
        print(f"✓ Encrypted {len(data)} bytes → {len(encrypted)} bytes")
        return encrypted
    
    def decrypt(self, encrypted_data: bytes) -> bytes:
        """
        Decrypt data.
        
        Args:
            encrypted_data: Data to decrypt
        
        Returns:
            bytes: Decrypted data
        """
        decrypted = self.cipher.decrypt(encrypted_data)
        print(f"✓ Decrypted {len(encrypted_data)} bytes → {len(decrypted)} bytes")
        return decrypted


class PasswordHasher:
    """
    Securely hashes passwords using bcrypt.
    """
    
    @staticmethod
    def hash_password(password: str) -> str:
        """
        Hash a password.
        
        Args:
            password: Password to hash
        
        Returns:
            str: Hashed password
        """
        import bcrypt
        salt = bcrypt.gensalt()
        hashed = bcrypt.hashpw(password.encode(), salt)
        return hashed.decode()
    
    @staticmethod
    def verify_password(password: str, hashed_password: str) -> bool:
        """
        Verify a password against a hash.
        
        Args:
            password: Password to verify
            hashed_password: Hashed password
        
        Returns:
            bool: True if password matches
        """
        import bcrypt
        return bcrypt.checkpw(password.encode(), hashed_password.encode())


# Example usage
print("=== Data Encryption Demo ===\n")

# Encrypt/decrypt sensitive data
print("--- Encrypting Sensitive Data ---\n")
password = b"my-secure-password-123"
encryptor = DataEncryptor(password)

sensitive_data = b"This is sensitive information: SSN 123-45-6789"
print(f"Original: {sensitive_data.decode()}\n")

encrypted = encryptor.encrypt(sensitive_data)
print(f"Encrypted: {encrypted[:50]}...\n")

decrypted = encryptor.decrypt(encrypted)
print(f"Decrypted: {decrypted.decode()}\n")

# Password hashing
print("--- Password Hashing ---\n")
password = "user_password_123"

hashed = PasswordHasher.hash_password(password)
print(f"Original password: {password}")
print(f"Hashed password: {hashed}\n")

# Verify password
correct = PasswordHasher.verify_password(password, hashed)
print(f"Password verification (correct): {'✓ Valid' if correct else '✗ Invalid'}\n")

wrong = PasswordHasher.verify_password("wrong_password", hashed)
print(f"Password verification (wrong): {'✓ Valid' if wrong else '✗ Invalid'}")
```

**Output:**
```
=== Data Encryption Demo ===

--- Encrypting Sensitive Data ---

Original: This is sensitive information: SSN 123-45-6789

✓ Encrypted 60 bytes → 120 bytes
Encrypted: gAAAAABkZ4mWj5XqY7tR8nN2mP4kL6vQ9wR2sT5u...

✓ Decrypted 120 bytes → 60 bytes
Decrypted: This is sensitive information: SSN 123-45-6789

--- Password Hashing ---

Original password: user_password_123
Hashed password: $2b$12$L9v8X2nN5mK7pQ3rR6tS8uW4yZ0bC1dE2fG3hI4jK5lM6nO7pQ8r

✓ Password verification (correct): ✓ Valid

✓ Password verification (wrong): ✗ Invalid
```

### **Key Management**

Proper key management is critical for encryption security.

```
┌─────────────────────────────────────────────────────────────────┐
│              KEY MANAGEMENT BEST PRACTICES                      │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Key Generation:                                                │
│  ✓ Use cryptographically secure random number generators         │
│  ✓ Use appropriate key lengths (AES-256, RSA-4096)              │
│  ✓ Generate unique keys per use case                            │
│                                                                 │
│  Key Storage:                                                   │
│  ✓ Never store keys in code                                    │
│  ✓ Use environment variables or secret managers                 │
│  ✓ Encrypt keys at rest                                         │
│  ✓ Use Hardware Security Modules (HSMs) for high security       │
│                                                                 │
│  Key Rotation:                                                  │
│  ✓ Rotate keys regularly (every 90 days)                       │
│  ✓ Use key versioning                                           │
│  ✓ Plan for rotation in advance                                │
│  ✓ Document rotation procedures                                 │
│                                                                 │
│  Key Access:                                                    │
│  ✓ Limit access to keys (principle of least privilege)          │
│  ✓ Log all key access                                           │
│  ✓ Audit key usage regularly                                    │
│  ✓ Revoke keys immediately if compromised                       │
│                                                                 │
│  Key Destruction:                                               │
│  ✓ Securely delete keys when no longer needed                   │
│  ✓ Use multi-pass overwriting                                   │
│  ✓ Verify destruction                                           │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

---

## **Common Vulnerabilities and Prevention**

### **SQL Injection**

SQL injection occurs when untrusted user input is included in SQL queries.

**The Problem:**

```python
# VULNERABLE CODE - DO NOT USE
def get_user(username):
    query = f"SELECT * FROM users WHERE username = '{username}'"
    return db.execute(query)

# Attacker sends: username = "admin' OR '1'='1"
# Resulting query:
# SELECT * FROM users WHERE username = 'admin' OR '1'='1'
# Returns all users! Attacker gains admin access!
```

**The Solution - Parameterized Queries:**

```python
# SECURE CODE - USE THIS
def get_user(username):
    query = "SELECT * FROM users WHERE username = %s"
    return db.execute(query, (username,))

# Attacker sends: username = "admin' OR '1'='1"
# Resulting query (with parameterization):
# SELECT * FROM users WHERE username = 'admin' OR '1'='1'
# But the input is treated as a literal string, not SQL code!
# Returns no users (correct behavior)
```

### **Cross-Site Scripting (XSS)**

XSS occurs when untrusted data is included in web pages without proper escaping.

**The Problem:**

```html
<!-- VULNERABLE CODE - DO NOT USE -->
<div>
  Welcome, {{ user_input }}!
</div>

<!-- Attacker sends: user_input = "<script>alert('XSS')</script>" -->
<!-- Resulting HTML: -->
<div>
  Welcome, <script>alert('XSS')</script>!
</div>
<!-- Script executes! Attacker can steal cookies, redirect, etc. -->
```

**The Solution - Output Encoding:**

```html
<!-- SECURE CODE - USE THIS -->
<div>
  Welcome, {{ escape(user_input) }}!
</div>

<!-- Attacker sends: user_input = "<script>alert('XSS')</script>" -->
<!-- Resulting HTML (with escaping): -->
<div>
  Welcome, &lt;script&gt;alert('XSS')&lt;/script&gt;!
</div>
<!-- Script is displayed as text, not executed! -->
```

### **Cross-Site Request Forgery (CSRF)**

CSRF tricks users into performing actions they didn't intend.

**The Problem:**

```
1. User logs into bank.example.com
2. Attacker sends user email with malicious link:
   <img src="https://bank.example.com/transfer?to=attacker&amount=1000">
3. User clicks link (or image loads automatically)
4. Browser sends authentication cookies with request
5. Bank transfers $1000 to attacker!
```

**The Solution - CSRF Tokens:**

```
1. Server generates unique CSRF token for each session
2. Token included in forms as hidden field
3. On form submit, server verifies token matches session
4. Attacker cannot guess token, cannot forge requests
```

```python
# CSRF Token Example
import secrets

def generate_csrf_token():
    """Generate a random CSRF token."""
    return secrets.token_hex(32)

def verify_csrf_token(session_token, form_token):
    """Verify CSRF token."""
    return secrets.compare_digest(session_token, form_token)

# Usage
session_csrf_token = generate_csrf_token()
# Store in session: session['csrf_token'] = session_csrf_token

# In HTML form:
# <input type="hidden" name="csrf_token" value="{{ session_csrf_token }}">

# On form submit:
# form_token = request.form.get('csrf_token')
# session_token = session.get('csrf_token')
# if verify_csrf_token(session_token, form_token):
#     # Process form
# else:
#     # Reject form
```

### **Denial of Service (DoS) Prevention**

DoS attacks aim to make your service unavailable.

**DoS Mitigation Strategies:**

```
┌─────────────────────────────────────────────────────────────────┐
│              DoS MITIGATION STRATEGIES                           │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  1. Rate Limiting                                               │
│     • Limit requests per IP/user                                │
│     • Use token bucket or leaky bucket                          │
│     • Implement at multiple levels (app, server, network)       │
│                                                                 │
│  2. Resource Limits                                             │
│     • Limit CPU, memory, disk usage per request                  │
│     • Use timeouts for all external calls                       │
│     • Implement request size limits                             │
│                                                                 │
│  3. Caching                                                     │
│     • Cache expensive computations                              │
│     • Use CDN for static content                                │
│     • Implement query result caching                            │
│                                                                 │
│  4. Circuit Breakers                                            │
│     • Stop calling failing dependencies                         │
│     • Implement graceful degradation                            │
│     • Provide fallback responses                                │
│                                                                 │
│  5. Scalability                                                 │
│     • Use horizontal scaling                                     │
│     • Implement auto-scaling                                    │
│     • Distribute across multiple regions                        │
│                                                                 │
│  6. DDoS Protection Services                                    │
│     • Use Cloudflare, AWS Shield, etc.                          │
│     • Implement rate limiting at edge                            │
│     • Use anycast routing                                       │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

---

## **Zero Trust Architecture**

Zero Trust is a security model that assumes no implicit trust.

### **The Zero Trust Principle**

```
Traditional Security Model:
  "Trust, but verify"
  • Inside network = trusted
  • Outside network = untrusted
  • Perimeter-based security

Zero Trust Model:
  "Never trust, always verify"
  • No implicit trust, regardless of location
  • Verify every request, every time
  • Least privilege access
  • Assume breach
```

### **Zero Trust Principles**

```
┌─────────────────────────────────────────────────────────────────┐
│              ZERO TRUST PRINCIPLES                               │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  1. Verify Explicitly                                           │
│     • Always authenticate and authorize                         │
│     • Use multi-factor authentication                           │
│     • Validate device health and posture                        │
│                                                                 │
│  2. Use Least Privilege Access                                  │
│     • Grant only necessary access                               │
│     • Just-in-time access                                       │
│     • Just-enough access                                       │
│     • Revoke access when no longer needed                      │
│                                                                 │
│  3. Assume Breach                                               │
│     • Design for containment                                    │
│     • Implement microsegmentation                               │
│     • Monitor and detect anomalies                              │
│     • Have incident response ready                              │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### **Zero Trust Architecture**

```
┌─────────────────────────────────────────────────────────────────┐
│              ZERO TRUST ARCHITECTURE                             │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│                   User/Device                                   │
│                       │                                         │
│                       │ 1. Authentication                       │
│                       │    (MFA, device trust)                   │
│                       ↓                                         │
│              ┌───────────────┐                                  │
│              │ Policy Engine  │                                  │
│              │               │                                  │
│              │ • User        │                                  │
│              │ • Device      │                                  │
│              │ • Location    │                                  │
│              │ • Data        │                                  │
│              │ • App         │                                  │
│              └───────┬───────┘                                  │
│                      │                                          │
│                      │ 2. Authorization                        │
│                      │    (RBAC, ABAC)                           │
│                      ↓                                          │
│              ┌───────────────┐                                  │
│              │   PDP         │ ← Policy Decision Point           │
│              │               │                                  │
│              │ Allow/Deny    │                                  │
│              └───────┬───────┘                                  │
│                      │                                          │
│        ┌─────────────┼─────────────┐                           │
│        ↓             ↓             ↓                           │
│  ┌──────────┐ ┌──────────┐ ┌──────────┐                        │
│  │ Resource │ │ Resource │ │ Resource │                        │
│  │   A      │ │   B      │ │   C      │                        │
│  └──────────┘ └──────────┘ └──────────┘                        │
│        │             │             │                            │
│        │ 3. Monitoring & Auditing                               │
│        │    • Log all access                                    │
│        │    • Detect anomalies                                   │
│        │    • Continuous validation                             │
│        ↓                                                         │
│  ┌──────────────┐                                               │
│  │   SIEM/SOAR  │ ← Security Information & Event Management    │
│  └──────────────┘                                               │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

---

## **Chapter Summary**

### **Key Takeaways**

| Concept | Summary |
|---------|---------|
| **Authentication** | Proves identity using factors (knowledge, possession, biometrics). JWT and OAuth 2.0 are standards for distributed systems |
| **Authorization** | Controls what authenticated users can do. RBAC (role-based) is simple; ABAC (attribute-based) is flexible |
| **API Security** | Protect endpoints with HTTPS, authentication, rate limiting, input validation, and proper error handling |
| **Data Protection** | Encrypt data at rest (AES-256) and in transit (TLS). Use proper key management |
| **Common Vulnerabilities** | Prevent SQL injection (parameterized queries), XSS (output encoding), CSRF (tokens), and DoS (rate limiting) |
| **Zero Trust** | Never trust, always verify. Use least privilege, assume breach, implement continuous validation |

### **Security Checklist**

Use this checklist when securing your systems:

```
Authentication & Authorization:
□ Multi-factor authentication implemented
□ JWT/OAuth 2.0 for distributed systems
□ Strong password policies (minimum 12 chars, complexity)
□ RBAC/ABAC for authorization
□ Regular permission audits

API Security:
□ HTTPS only (TLS 1.2+)
□ API keys for service-to-service
□ Rate limiting implemented
□ Input validation on all endpoints
□ Output encoding for XSS prevention
□ CSRF tokens for state-changing operations

Data Protection:
□ Encryption at rest (AES-256)
□ Encryption in transit (TLS)
□ Passwords hashed (bcrypt/argon2)
□ Sensitive data not logged
□ Key management system in place

Infrastructure:
□ Security headers (CSP, HSTS, X-Frame-Options)
□ Web Application Firewall (WAF)
□ DDoS protection
⬯ Regular security updates
⬯ Vulnerability scanning

Monitoring & Auditing:
□ All access logged
□ Anomaly detection implemented
□ Security incident response plan
□ Regular penetration testing
□ Security awareness training

Zero Trust:
□ Verify explicitly (identity, device, location)
□ Least privilege access
□ Assume breach mentality
□ Microsegmentation
□ Continuous monitoring
```

### **Common Mistakes to Avoid**

| Mistake | Why It's Bad | How to Fix It |
|---------|--------------|---------------|
| Storing passwords in plain text | Any breach exposes all passwords | Use bcrypt/argon2 to hash passwords |
| Hardcoding secrets in code | Secrets visible to anyone with code access | Use environment variables or secret managers |
| Using HTTP instead of HTTPS | Data can be intercepted | Always use HTTPS/TLS |
| SQL injection vulnerabilities | Attackers can read/modify/delete data | Use parameterized queries |
| Not validating input | Invalid data can cause security issues | Validate all input on server-side |
| Trusting client-side validation | Attackers can bypass it | Always validate on server-side |
| Not implementing rate limiting | DoS attacks easy | Implement rate limiting at multiple levels |
| Logging sensitive data | Logs can expose secrets | Never log passwords, tokens, PII |
| Not rotating secrets | Compromised secrets remain valid | Rotate secrets regularly |
| Ignoring security updates | Known vulnerabilities remain exploitable | Keep dependencies updated |

---

## **Exercises**

### **Exercise 1: Implement JWT Authentication**

Create a simple API with JWT authentication:
- Login endpoint that returns JWT
- Protected endpoint that requires valid JWT
- Token expiration handling
- Refresh token mechanism

### **Exercise 2: Design RBAC System**

Design an RBAC system for a content management system:
- Define roles (Admin, Editor, Author, Viewer)
- Define permissions for each role
- Implement role assignment
- Implement permission checking

### **Exercise 3: Secure an API**

Given a vulnerable API, identify and fix security issues:
- SQL injection
- XSS vulnerability
- Missing authentication
- No rate limiting
- Sensitive data in logs

---

## **Further Reading**

| Resource | Description |
|----------|-------------|
| [OWASP Top 10](https://owasp.org/www-project-top-ten/) | Most critical web application security risks |
| [JWT.io](https://jwt.io/) | JWT documentation and tools |
| [OAuth 2.0 Specification](https://oauth.net/2/) | Official OAuth 2.0 specification |
| [Zero Trust Architecture](https://www.microsoft.com/en-us/security/zero-trust) | Microsoft's Zero Trust guidance |
| [NIST Cybersecurity Framework](https://www.nist.gov/cyberframework) | Industry-standard security framework |

---

**Next:** In Chapter 13, we'll explore **Monolithic vs. Microservices Architecture**, discussing when to use each approach, how to decompose monoliths, and the challenges of microservices including service discovery, configuration management, and the Strangler Fig pattern for migration.


<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='11. reliability_and_fault_tolerance.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='../5. Architectural_patterns/13. monolithic_vs_microservices.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
